In [ ]:
!pip -q install wandb

import os, math, time, contextlib, copy
from dataclasses import dataclass, asdict
from typing import Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import amp
from tqdm import trange

import wandb
wandb.login()

# ---- Speed knobs ----
def enable_tf32():
    if torch.cuda.is_available():
        if hasattr(torch.backends.cuda.matmul, "fp32_precision"):
            torch.backends.cuda.matmul.fp32_precision = "tf32"
        else:
            torch.backends.cuda.matmul.allow_tf32 = True

        if (
            hasattr(torch.backends, "cudnn")
            and hasattr(torch.backends.cudnn, "conv")
            and hasattr(torch.backends.cudnn.conv, "fp32_precision")
        ):
            torch.backends.cudnn.conv.fp32_precision = "tf32"
        else:
            torch.backends.cudnn.allow_tf32 = True

    try:
        torch.set_float32_matmul_precision("high")
    except Exception:
        pass

enable_tf32()

def flash_sdpa_context():
    """Request FlashAttention kernels when available; safe fallback otherwise."""
    try:
        from torch.nn.attention import sdpa_kernel, SDPBackend
        return sdpa_kernel([SDPBackend.FLASH_ATTENTION])
    except Exception:
        return contextlib.nullcontext()


wandb: WARNING Path ./wandb_grid_bs8192_20260106_221332/wandb/ wasn't writable, using system temp directory


In [ ]:
import os, time, copy, math
from dataclasses import dataclass, asdict
from typing import Optional, Tuple, Iterable
from fractions import Fraction
from contextlib import nullcontext

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import trange
import wandb


# -------------------------
# Robust context helpers (no device_type kwarg anywhere)
# -------------------------
def get_sdpa_context():
    """
    Returns a real context-manager object (not a generator).
    Uses torch.nn.attention.sdpa_kernel if available; otherwise no-op.
    """
    try:
        from torch.nn.attention import sdpa_kernel, SDPBackend
        return sdpa_kernel([SDPBackend.FLASH_ATTENTION, SDPBackend.EFFICIENT_ATTENTION, SDPBackend.MATH])
    except Exception:
        return nullcontext()

def get_autocast_context(device: torch.device, dtype: Optional[torch.dtype]):
    """
    Returns a real context-manager object for autocast.
    Never passes device_type=... (compat across torch versions).
    """
    if device.type != "cuda" or dtype is None:
        return nullcontext()

    # Prefer torch.amp.autocast if present; else fallback to torch.cuda.amp.autocast
    try:
        return torch.amp.autocast("cuda", dtype=dtype)          # positional device arg
    except Exception:
        return torch.cuda.amp.autocast(dtype=dtype)             # no device_type kwarg

def make_grad_scaler(enabled: bool):
    """
    Returns a scaler-like object with .is_enabled(), .scale(), .step(), .update().
    """
    if not enabled:
        class _NoScaler:
            def is_enabled(self): return False
            def scale(self, x): return x
            def step(self, opt): opt.step()
            def update(self): pass
        return _NoScaler()

    # Prefer torch.amp.GradScaler if present; else torch.cuda.amp.GradScaler
    try:
        return torch.amp.GradScaler()
    except Exception:
        return torch.cuda.amp.GradScaler()


# -------------------------
# Model components
# -------------------------
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.fc_1 = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.gelu = nn.GELU(approximate="tanh")
        self.fc_2 = nn.Linear(3 * config.n_embd, config.n_embd)

    def forward(self, x):
        return self.fc_2(self.gelu(self.fc_1(x)))


class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_heads == 0
        self.n_embd = config.n_embd
        self.n_heads = config.n_heads
        self.head_dim = config.n_embd // config.n_heads

        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = True

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)

        q = q.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)

        y = F.scaled_dot_product_attention(q, k, v, dropout_p=0.0, is_causal=True)
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.c_proj(y)
        return y


class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.use_mlp = bool(getattr(config, "use_mlp", True))

        self.attn = CausalSelfAttention(config)
        self.ln_1 = nn.LayerNorm(config.n_embd)

        if self.use_mlp:
            self.mlp = MLP(config)
            self.ln_2 = nn.LayerNorm(config.n_embd)
        else:
            self.mlp = None
            self.ln_2 = None

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        if self.mlp is not None:
            x = x + self.mlp(self.ln_2(x))
        return x


class GPTConfig:
    def __init__(
        self,
        block_size: int,
        vocab_size: int,
        n_layers: int = 2,
        n_heads: int = 1,
        n_embd: int = 64,
        without_pos: bool = False,
        use_mlp: bool = True,                 # NEW
        max_seq_len: Optional[int] = None,
    ):
        self.block_size = int(block_size)
        self.vocab_size = int(vocab_size)
        self.n_layers = int(n_layers)
        self.n_heads = int(n_heads)
        self.n_embd = int(n_embd)
        self.without_pos = bool(without_pos)
        self.use_mlp = bool(use_mlp)           # NEW
        self.max_seq_len = int(max_seq_len if max_seq_len is not None else (2 * self.block_size + 1))


class GPT(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.config = config
        self.n_layers = config.n_layers

        self.transformer = nn.ModuleDict(
            dict(
                wte=nn.Embedding(config.vocab_size, config.n_embd),
                wpe=nn.Embedding(config.max_seq_len, config.n_embd),
                h=nn.ModuleList([Block(config) for _ in range(config.n_layers)]),
                ln_f=nn.LayerNorm(config.n_embd),
            )
        )
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.lm_head.weight = self.transformer.wte.weight  # weight tying
        self.apply(self._init_weights)

        self.register_buffer("pos_idx", torch.arange(config.max_seq_len), persistent=False)

        # Positional encodings fixed to 0 (and frozen) if without_pos=True
        if self.config.without_pos:
            with torch.no_grad():
                self.transformer.wpe.weight.zero_()
            self.transformer.wpe.weight.requires_grad_(False)

    def _init_weights(self, module):
        std = 0.02
        if isinstance(module, nn.Linear):
            if hasattr(module, "NANOGPT_SCALE_INIT"):
                std *= (2 * self.n_layers) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)

    def forward(self, idx, return_full_logits: bool = False, block_size: Optional[int] = None):
        B, T = idx.size()
        if block_size is None:
            block_size = self.config.block_size
        block_size = int(block_size)

        expected_T = 2 * block_size + 1
        assert T == expected_T, f"Expected T={expected_T} for block_size={block_size}, got T={T}"
        assert T <= self.config.max_seq_len, f"T={T} exceeds max_seq_len={self.config.max_seq_len}"

        pos = self.transformer.wpe(self.pos_idx[:T])
        x = self.transformer.wte(idx) if self.config.without_pos else (self.transformer.wte(idx) + pos)

        for block in self.transformer.h:
            x = block(x)

        x = self.transformer.ln_f(x)

        targets = idx[:, block_size + 1 :]  # (B, K)

        if return_full_logits:
            logits = self.lm_head(x)                       # (B, T, V)
            logits_for_loss = logits[:, block_size:T-1, :] # (B, K, V)
        else:
            x_for_loss = x[:, block_size:T-1, :]           # (B, K, C)
            logits_for_loss = self.lm_head(x_for_loss)     # (B, K, V)
            logits = logits_for_loss

        loss = F.cross_entropy(
            logits_for_loss.reshape(-1, logits_for_loss.size(-1)),
            targets.reshape(-1),
        )
        return logits, loss


# -------------------------
# Data batching
# -------------------------
SEP_TOKEN = "SEP"

def _sample_numbers(batch_size: int, vocab_n: int, block_size: int, device: torch.device, allow_duplicates: bool):
    if allow_duplicates:
        return torch.randint(0, vocab_n, (batch_size, block_size), device=device, dtype=torch.long)

    if block_size > vocab_n:
        raise ValueError(f"allow_duplicates=False requires block_size <= vocab_n (got block_size={block_size}, vocab_n={vocab_n})")

    scores = torch.rand(batch_size, vocab_n, device=device)
    return scores.topk(block_size, dim=1).indices.to(torch.long)

def get_batch(batch_size: int, device: torch.device, vocab_n: int, block_size: int, allow_duplicates: bool):
    x = _sample_numbers(batch_size, vocab_n, block_size, device, allow_duplicates)
    vals = x.sort(dim=1).values
    sep_id = vocab_n
    sep = torch.full((batch_size, 1), sep_id, device=device, dtype=torch.long)
    return torch.cat([x, sep, vals], dim=1)


# -------------------------
# LR + plateau helpers
# -------------------------
def create_optimizer(model, weight_decay: float, lr: float):
    params = [p for p in model.parameters() if p.requires_grad]
    decay_params = [p for p in params if p.dim() > 1]
    nondecay_params = [p for p in params if p.dim() <= 1]
    optim_groups = [
        {"params": decay_params, "weight_decay": weight_decay},
        {"params": nondecay_params, "weight_decay": 0.0},
    ]
    try:
        return torch.optim.AdamW(optim_groups, lr=lr, betas=(0.9, 0.95), eps=1e-8, fused=True)
    except TypeError:
        return torch.optim.AdamW(optim_groups, lr=lr, betas=(0.9, 0.95), eps=1e-8)

def get_lr(itr: int, cfg) -> float:
    if itr < cfg.warmup_iters:
        return cfg.learning_rate * (itr + 1) / (cfg.warmup_iters + 1)
    if itr > cfg.max_iters:
        return cfg.min_lr
    ratio = (itr - cfg.warmup_iters) / (cfg.max_iters - cfg.warmup_iters)
    ratio = 0.5 * (1.0 + math.cos(math.pi * ratio))
    return cfg.min_lr + ratio * (cfg.learning_rate - cfg.min_lr)

def _safe_log10(x: float, eps: float = 1e-30) -> float:
    return math.log10(max(float(x), eps))

def _linreg_slope(xs, ys) -> float:
    n = len(xs)
    if n < 2:
        return 0.0
    x_mean = sum(xs) / n
    y_mean = sum(ys) / n
    cov = 0.0
    var = 0.0
    for x, y in zip(xs, ys):
        dx = x - x_mean
        dy = y - y_mean
        cov += dx * dy
        var += dx * dx
    return cov / var if var > 0 else 0.0


# -------------------------
# Ratio + embedding dim helpers
# -------------------------
def _ratio_to_tag(r: Optional[float]) -> str:
    if r is None:
        return "none"
    fr = Fraction(float(r)).limit_denominator(1024)
    return f"{fr.numerator}over{fr.denominator}"

def _block_sizes_to_tag(ks: Tuple[int, ...]) -> str:
    return "K" + "-".join(str(int(k)) for k in ks)

def compute_effective_n_embd(vocab_n: int, n_heads: int, vocab_over_embd: Optional[float], fallback_n_embd: int) -> int:
    """
    Implements: (vocab_size / embd_dim) = r  => embd_dim = vocab_size / r
    vocab_size includes SEP => total_vocab = vocab_n + 1
    """
    if vocab_over_embd is None:
        n_embd = int(fallback_n_embd)
    else:
        r = float(vocab_over_embd)
        if r <= 0:
            raise ValueError(f"vocab_over_embd must be > 0, got {vocab_over_embd}")
        total_vocab_size = int(vocab_n) + 1
        n_embd = int(round(total_vocab_size / r))
        n_embd = max(1, n_embd)

    n_heads = int(n_heads)
    if n_embd % n_heads != 0:
        n_embd = ((n_embd + n_heads - 1) // n_heads) * n_heads
    return int(n_embd)

def acc_from_logits(logits: torch.Tensor, idx: torch.Tensor, block_size: int):
    targets = idx[:, block_size + 1 :]  # (B, K)
    preds = logits.argmax(dim=-1)       # (B, K)
    token_acc = (preds == targets).float().mean()
    sample_acc = (preds == targets).all(dim=1).float().mean()
    return token_acc, sample_acc

@torch.no_grad()
def eval_batch_metrics(model: nn.Module, device: torch.device, cfg, block_size: int, use_amp: bool, amp_dtype: Optional[torch.dtype], batch_size: Optional[int] = None):
    model.eval()
    bs = int(batch_size) if batch_size is not None else int(cfg.micro_batch_size)

    batch = get_batch(
        batch_size=bs,
        device=device,
        vocab_n=cfg.vocab_n,
        block_size=int(block_size),
        allow_duplicates=cfg.allow_duplicates,
    )

    with (get_autocast_context(device, amp_dtype) if use_amp else nullcontext()):
        logits, loss = model(batch, return_full_logits=False, block_size=int(block_size))

    token_acc_t, sample_acc_t = acc_from_logits(logits, batch, int(block_size))
    model.train()
    return float(loss.item()), float(token_acc_t.item()), float(sample_acc_t.item())


# -------------------------
# Config
# -------------------------
@dataclass
class TrainConfig:
    vocab_n: int = 1024
    block_size: int = 32
    allow_duplicates: bool = True
    sep_token: str = SEP_TOKEN

    test_block_sizes: Optional[Tuple[int, ...]] = None

    n_layers: int = 2
    n_heads: int = 1
    n_embd: int = 64
    vocab_over_embd: Optional[float] = None

    without_pos: bool = False
    use_mlp: bool = True  # NEW: default uses MLPs

    warmup_iters: int = 200
    max_iters: int = 120000
    learning_rate: float = 1e-4
    min_lr: float = 1e-6
    weight_decay: float = 0.0

    micro_batch_size: int = 1024
    effective_batch_size: int = 4096

    log_interval: int = 250
    ckpt_interval: int = 20000
    save_dir: str = "./saved_models"

    plateau_window_logs: int = 40
    plateau_slope_threshold: float = 0.02
    plateau_log10_loss_delta: float = 0.02
    plateau_patience_logs: int = 2
    plateau_extra_logs: int = 1
    plateau_min_iters: int = 20000

    seed: int = 1337
    use_compile: bool = False

    wandb_project: str = "sortgpt"
    wandb_entity: Optional[str] = None
    wandb_group: Optional[str] = None
    wandb_mode: Optional[str] = None


# -------------------------
# Naming / saving
# -------------------------
def make_wandb_run_name(cfg: TrainConfig) -> str:
    rtag = _ratio_to_tag(cfg.vocab_over_embd)
    test_ks = cfg.test_block_sizes if (cfg.test_block_sizes and len(cfg.test_block_sizes) > 0) else (cfg.block_size,)
    ktag = _block_sizes_to_tag(tuple(int(k) for k in test_ks))
    return (
        f"sortgpt"
        f"_N{int(cfg.vocab_n)}"
        f"_K{int(cfg.block_size)}"
        f"_L{int(cfg.n_layers)}"
        f"_npos{int(cfg.without_pos)}"
        f"_mlp{int(cfg.use_mlp)}"
        f"_r{rtag}"
        f"_test{ktag}"
        f"_dup{int(cfg.allow_duplicates)}"
    )

def make_save_filename(prefix: str, cfg: TrainConfig, iters_done: int) -> str:
    eff_n_embd = compute_effective_n_embd(cfg.vocab_n, cfg.n_heads, cfg.vocab_over_embd, cfg.n_embd)
    test_ks = cfg.test_block_sizes if (cfg.test_block_sizes and len(cfg.test_block_sizes) > 0) else (cfg.block_size,)
    test_ks = tuple(int(k) for k in test_ks)
    rtag = _ratio_to_tag(cfg.vocab_over_embd)
    ktag = _block_sizes_to_tag(test_ks)
    return (
        f"{prefix}"
        f"_N{int(cfg.vocab_n)}"
        f"_K{int(cfg.block_size)}"
        f"_L{int(cfg.n_layers)}"
        f"_H{int(cfg.n_heads)}"
        f"_E{int(eff_n_embd)}"
        f"_r{rtag}"
        f"_npos{int(cfg.without_pos)}"
        f"_mlp{int(cfg.use_mlp)}"
        f"_dup{int(cfg.allow_duplicates)}"
        f"_test{ktag}"
        f"_iters{int(iters_done)}.pt"
    )


# -------------------------
# Training
# -------------------------
def train_sorting_gpt(cfg: TrainConfig):
    torch.manual_seed(cfg.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(cfg.seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"using device: {device}")

    # Default test sizes if not specified
    test_block_sizes = tuple(int(k) for k in (cfg.test_block_sizes if cfg.test_block_sizes else (cfg.block_size,)))

    # Validate no-duplicates across train + test K
    if not cfg.allow_duplicates:
        for k in (int(cfg.block_size),) + tuple(test_block_sizes):
            if k > cfg.vocab_n:
                raise ValueError(f"allow_duplicates=False requires block_size <= vocab_n (got block_size={k}, vocab_n={cfg.vocab_n})")
        if cfg.block_size == cfg.vocab_n:
            print("⚠️  Warning: block_size == vocab_n with allow_duplicates=False => degenerate constant sorted output.")

    total_vocab_size = cfg.vocab_n + 1
    sep_id = cfg.vocab_n

    grad_accum_steps = cfg.effective_batch_size // cfg.micro_batch_size
    assert cfg.effective_batch_size % cfg.micro_batch_size == 0

    # AMP settings
    use_amp = (device.type == "cuda")
    if use_amp:
        bf16_ok = getattr(torch.cuda, "is_bf16_supported", lambda: False)()
        amp_dtype = torch.bfloat16 if bf16_ok else torch.float16
    else:
        amp_dtype = None

    scaler = make_grad_scaler(enabled=(use_amp and amp_dtype == torch.float16))

    # Effective embd dim via ratio; max_seq_len for largest tested K
    eff_n_embd = compute_effective_n_embd(cfg.vocab_n, cfg.n_heads, cfg.vocab_over_embd, cfg.n_embd)
    max_k_for_model = max([int(cfg.block_size)] + [int(k) for k in test_block_sizes])
    max_seq_len = 2 * max_k_for_model + 1

    model_cfg = GPTConfig(
        block_size=cfg.block_size,
        vocab_size=total_vocab_size,
        n_layers=cfg.n_layers,
        n_heads=cfg.n_heads,
        n_embd=eff_n_embd,
        without_pos=cfg.without_pos,
        use_mlp=cfg.use_mlp,
        max_seq_len=max_seq_len,
    )
    model = GPT(model_cfg).to(device)

    if cfg.use_compile and hasattr(torch, "compile"):
        try:
            model = torch.compile(model, mode="max-autotune")
            print("torch.compile enabled")
        except Exception as e:
            print(f"torch.compile failed, continuing uncompiled: {e}")

    optimizer = create_optimizer(model, weight_decay=cfg.weight_decay, lr=cfg.learning_rate)
    os.makedirs(cfg.save_dir, exist_ok=True)

    if wandb.run is not None:
        wandb.finish()

    wandb_cfg = asdict(cfg)
    wandb_cfg.update(
        dict(
            total_vocab_size=total_vocab_size,
            sep_id=sep_id,
            max_seq_len=max_seq_len,
            max_k_for_model=max_k_for_model,
            grad_accum_steps=grad_accum_steps,
            amp_dtype=str(amp_dtype) if amp_dtype is not None else "none",
            device=str(device),
            test_block_sizes=list(test_block_sizes),
            n_embd_effective=int(eff_n_embd),
        )
    )

    run = wandb.init(
        project=cfg.wandb_project,
        entity=cfg.wandb_entity,
        group=cfg.wandb_group,
        name=make_wandb_run_name(cfg),
        config=wandb_cfg,
        mode=cfg.wandb_mode,
    )

    # Metrics
    run.define_metric("iter")
    run.define_metric("train/*", step_metric="iter")
    run.define_metric("test/*", step_metric="iter")
    run.define_metric("lr", step_metric="iter")
    run.define_metric("plateau/*", step_metric="iter")
    run.define_metric("ll/iter")
    run.define_metric("ll/*", step_metric="ll/iter")

    ll_iters_hist, ll_loss_hist = [], []
    plateau_hits = 0
    plateau_reached = False
    plateau_reached_iter = None
    stop_after_log_iter = None

    last_logged_itr = None
    iters_done = cfg.max_iters
    last_log_t = time.time()

    with get_sdpa_context():
        for itr in trange(cfg.max_iters, desc="training"):
            optimizer.zero_grad(set_to_none=True)
            loss_accum = torch.zeros((), device=device)

            do_log = (itr % cfg.log_interval == 0)
            if do_log:
                token_correct = torch.zeros((), device=device)
                sample_correct = torch.zeros((), device=device)
                token_total = 0
                sample_total = 0

            for _ in range(grad_accum_steps):
                batch = get_batch(
                    batch_size=cfg.micro_batch_size,
                    device=device,
                    vocab_n=cfg.vocab_n,
                    block_size=cfg.block_size,
                    allow_duplicates=cfg.allow_duplicates,
                )

                if use_amp:
                    with get_autocast_context(device, amp_dtype):
                        logits, loss = model(batch, return_full_logits=False, block_size=cfg.block_size)
                else:
                    logits, loss = model(batch, return_full_logits=False, block_size=cfg.block_size)

                # Train acc on the same microbatches at log steps
                if do_log:
                    with torch.no_grad():
                        targets = batch[:, cfg.block_size + 1 :]
                        preds = logits.detach().argmax(dim=-1)
                        token_correct += (preds == targets).sum()
                        sample_correct += (preds == targets).all(dim=1).sum()
                        token_total += targets.numel()
                        sample_total += targets.size(0)

                loss_to_back = loss / grad_accum_steps
                if scaler.is_enabled():
                    scaler.scale(loss_to_back).backward()
                else:
                    loss_to_back.backward()

                loss_accum += loss.detach()

            lr = get_lr(itr, cfg)
            for pg in optimizer.param_groups:
                pg["lr"] = lr

            if scaler.is_enabled():
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()

            # ---- log + plateau check ----
            if do_log:
                train_loss = float((loss_accum / grad_accum_steps).item())
                train_token_acc = float((token_correct / max(token_total, 1)).item())
                train_sample_acc = float((sample_correct / max(sample_total, 1)).item())

                steps_done = itr + 1
                ll_iter = _safe_log10(steps_done)
                ll_loss = _safe_log10(train_loss)
                ll_lr = _safe_log10(lr)

                ll_iters_hist.append(ll_iter)
                ll_loss_hist.append(ll_loss)

                slope = None
                delta_ll_loss = None
                plateau_now = 0

                if (not plateau_reached) and (len(ll_iters_hist) >= cfg.plateau_window_logs) and (steps_done >= cfg.plateau_min_iters):
                    xs = ll_iters_hist[-cfg.plateau_window_logs:]
                    ys = ll_loss_hist[-cfg.plateau_window_logs:]
                    slope = _linreg_slope(xs, ys)
                    delta_ll_loss = ys[0] - ys[-1]
                    is_improving = (delta_ll_loss >= 0.0)
                    plateau_now = int(
                        is_improving and (delta_ll_loss < cfg.plateau_log10_loss_delta) and (slope > -cfg.plateau_slope_threshold)
                    )
                    plateau_hits = (plateau_hits + 1) if plateau_now else 0
                    if plateau_hits >= cfg.plateau_patience_logs:
                        plateau_reached = True
                        plateau_reached_iter = itr
                        stop_after_log_iter = itr + cfg.log_interval * max(int(cfg.plateau_extra_logs), 0)
                        print(
                            f"🟨 plateau detected @ itr={itr} (slope={slope:.4f}, Δlog10(loss)={delta_ll_loss:.4f}); "
                            f"will stop after logging itr={stop_after_log_iter}"
                        )

                now = time.time()
                dt = now - last_log_t
                last_log_t = now
                print(
                    f"itr: {itr} lr: {lr:.3e} train loss: {train_loss:.6f} "
                    f"train token_acc: {train_token_acc:.4f} train sample_acc: {train_sample_acc:.4f} (dt={dt:.2f}s)"
                )

                log_dict = {
                    "iter": steps_done,
                    "train/loss": train_loss,
                    "train/token_acc": train_token_acc,
                    "train/sample_acc": train_sample_acc,
                    "lr": lr,
                    "ll/iter": ll_iter,
                    "ll/train_loss": ll_loss,
                    "ll/lr": ll_lr,
                    "plateau/now": plateau_now,
                    "plateau/hits": plateau_hits,
                    "plateau/reached": int(plateau_reached),
                }
                if slope is not None:
                    log_dict["plateau/loglog_slope"] = float(slope)
                if delta_ll_loss is not None:
                    log_dict["plateau/delta_log10_loss_window"] = float(delta_ll_loss)

                # Test metrics on each test block size
                for k in test_block_sizes:
                    test_loss, test_token_acc, test_sample_acc = eval_batch_metrics(
                        model=model,
                        device=device,
                        cfg=cfg,
                        block_size=int(k),
                        use_amp=use_amp,
                        amp_dtype=amp_dtype,
                        batch_size=cfg.micro_batch_size,
                    )
                    log_dict[f"test/K{int(k)}/loss"] = test_loss
                    log_dict[f"test/K{int(k)}/token_acc"] = test_token_acc
                    log_dict[f"test/K{int(k)}/sample_acc"] = test_sample_acc

                run.log(log_dict, step=steps_done)
                last_logged_itr = itr

                if plateau_reached and (stop_after_log_iter is not None) and (itr >= stop_after_log_iter):
                    iters_done = steps_done
                    break

            # ---- checkpointing ----
            if cfg.ckpt_interval and (itr > 0) and (itr % cfg.ckpt_interval == 0):
                steps_done = itr + 1
                ckpt_name = make_save_filename("Checkpoint", cfg, steps_done)
                ckpt_path = os.path.join(cfg.save_dir, ckpt_name)
                torch.save(
                    {
                        "itr": itr,
                        "iters_done": steps_done,
                        "plateau_reached": plateau_reached,
                        "plateau_reached_iter": plateau_reached_iter,
                        "train_config": wandb_cfg,
                        "model_config": vars(model_cfg),
                        "model": model.state_dict(),
                        "optimizer": optimizer.state_dict(),
                    },
                    ckpt_path,
                )
                print(f"saved checkpoint: {ckpt_path}")

    # Final save (filename includes grid params)
    final_name = make_save_filename("Final", cfg, iters_done)
    final_path = os.path.join(cfg.save_dir, final_name)
    torch.save(
        {
            "iters_done": iters_done,
            "plateau_reached": plateau_reached,
            "plateau_reached_iter": plateau_reached_iter,
            "train_config": wandb_cfg,
            "model_config": vars(model_cfg),
            "model": model.state_dict(),
        },
        final_path,
    )
    print(f"saved final model: {final_path}")

    run.summary["iters_done"] = iters_done
    run.summary["plateau_reached"] = bool(plateau_reached)
    run.summary["plateau_reached_iter"] = int(plateau_reached_iter) if plateau_reached_iter is not None else None
    run.finish()
    return final_path


# -------------------------
# Grid runner (NOW: allow_duplicates + use_mlp are grid dimensions)
# -------------------------
def run_grid(
    base_cfg: TrainConfig,
    vocab_sizes: Iterable[int],
    layer_counts: Iterable[int],
    block_sizes: Iterable[int],
    without_pos_flags: Iterable[bool],
    vocab_over_embd_list: Iterable[float],
    allow_duplicates_flags: Optional[Iterable[bool]] = None,  # NEW
    use_mlp_flags: Optional[Iterable[bool]] = None,           # NEW
):
    vocab_sizes = sorted(list(vocab_sizes), reverse=True)
    block_sizes = sorted(list(block_sizes), reverse=True)
    layer_counts = sorted(list(layer_counts), reverse=True)
    vocab_over_embd_list = sorted(list(vocab_over_embd_list))  # smaller => bigger embd => more expensive

    if allow_duplicates_flags is None:
        allow_duplicates_flags = [bool(base_cfg.allow_duplicates)]
    else:
        allow_duplicates_flags = list(allow_duplicates_flags)

    if use_mlp_flags is None:
        use_mlp_flags = [True]  # per your request: default is to use MLPs
    else:
        use_mlp_flags = list(use_mlp_flags)

    # Default test block sizes = largest block size in the grid (same as your request)
    if base_cfg.test_block_sizes is None or len(base_cfg.test_block_sizes) == 0:
        default_test_block_sizes = (max(int(k) for k in block_sizes),)
    else:
        default_test_block_sizes = tuple(int(k) for k in base_cfg.test_block_sizes)

    for N in vocab_sizes:
        for K in block_sizes:
            for L in layer_counts:
                for npos in without_pos_flags:
                    for mlp_on in use_mlp_flags:
                        for dup in allow_duplicates_flags:
                            for r in vocab_over_embd_list:
                                cfg = copy.deepcopy(base_cfg)
                                cfg.vocab_n = int(N)
                                cfg.block_size = int(K)
                                cfg.n_layers = int(L)
                                cfg.without_pos = bool(npos)
                                cfg.use_mlp = bool(mlp_on)
                                cfg.allow_duplicates = bool(dup)
                                cfg.vocab_over_embd = float(r)

                                cfg.test_block_sizes = default_test_block_sizes
                                train_sorting_gpt(cfg)


In [ ]:
import os, time

STAMP = time.strftime("%Y%m%d_%H%M%S")

PROJECT = globals().get("PROJECT", "sortgpt")
GROUP = f"grid_{STAMP}"

GRID_ROOT = f"./grid_outputs_{GROUP}"
WANDB_DIR = os.path.join(GRID_ROOT, "wandb")
SAVE_DIR  = os.path.join(GRID_ROOT, "saved_models")

os.makedirs(WANDB_DIR, exist_ok=True)
os.makedirs(SAVE_DIR, exist_ok=True)

# Make sure ALL wandb local artifacts go here
os.environ["WANDB_DIR"] = WANDB_DIR
os.environ["WANDB_CACHE_DIR"] = os.path.join(GRID_ROOT, "wandb_cache")
os.environ["WANDB_CONFIG_DIR"] = os.path.join(GRID_ROOT, "wandb_config")

print("GRID_ROOT =", GRID_ROOT)
print("WANDB_DIR =", WANDB_DIR)
print("SAVE_DIR  =", SAVE_DIR)
print("PROJECT   =", PROJECT)
print("GROUP     =", GROUP)

# Optional: nicer run name including all grid params
def make_wandb_run_name(cfg: TrainConfig) -> str:
    ga = int(cfg.effective_batch_size) // int(cfg.micro_batch_size)
    rtag = _ratio_to_tag(cfg.vocab_over_embd)
    test_ks = cfg.test_block_sizes if (cfg.test_block_sizes and len(cfg.test_block_sizes) > 0) else (cfg.block_size,)
    ktag = _block_sizes_to_tag(tuple(int(k) for k in test_ks))
    return (
        f"bs{int(cfg.micro_batch_size)}"
        f"_eb{int(cfg.effective_batch_size)}"
        f"_ga{ga}"
        f"_N{int(cfg.vocab_n)}"
        f"_K{int(cfg.block_size)}"
        f"_L{int(cfg.n_layers)}"
        f"_npos{int(cfg.without_pos)}"
        f"_mlp{int(cfg.use_mlp)}"
        f"_r{rtag}"
        f"_test{ktag}"
        f"_dup{int(cfg.allow_duplicates)}"
    )

base_cfg = TrainConfig(
    vocab_n=256,              # overridden by grid
    block_size=16,            # overridden by grid
    allow_duplicates=False,   # overridden via allow_duplicates_flags in run_grid

    n_layers=2,               # overridden by grid
    n_heads=1,

    n_embd=64,                # used only if vocab_over_embd is None (grid sets vocab_over_embd)
    vocab_over_embd=None,

    without_pos=False,        # overridden by grid
    use_mlp=True,             # default True (can be overridden via use_mlp_flags)
    test_block_sizes=None,

    max_iters=60000,
    warmup_iters=200,
    learning_rate=1e-4,
    min_lr=1e-6,
    weight_decay=0.0,

    micro_batch_size=4096,
    effective_batch_size=4096,

    log_interval=250,
    ckpt_interval=20000,
    save_dir=SAVE_DIR,

    plateau_window_logs=40,
    plateau_min_iters=20000,
    plateau_slope_threshold=0.02,
    plateau_log10_loss_delta=0.02,
    plateau_patience_logs=2,
    plateau_extra_logs=1,

    wandb_project=PROJECT,
    wandb_group=GROUP,
)

run_grid(
    base_cfg,
    vocab_sizes=[128, 64],
    layer_counts=[1, 2, 3],
    block_sizes=[16],
    without_pos_flags=[False, True],
    vocab_over_embd_list=[2, 4, 8],
    allow_duplicates_flags=[False],
    use_mlp_flags=[True],
)